In [6]:
import numpy as np
import PWBeamformer as pwb
import matplotlib.pyplot as plt
import trig

In [7]:
na = 11
nele = 128
dele = 0.298E-3

In [8]:
dx = dele
xmin = -10E-3
xmax = 10E-3
dy = 2E-3
ymin = 0E-3
ymax = 0E-3
dz = 150E-6/4
zmin = 1E-3
zmax = 40E-3

xfield = np.arange(xmin, xmax, dx)
yfield = 0
zfield = np.arange(zmin, zmax, dz)
Xfield, Yfield, Zfield = np.meshgrid(xfield, yfield, zfield)
field = np.array([Xfield.flatten(), Yfield.flatten(), Zfield.flatten()]).T

xele = dele * (np.arange(nele) - (nele-1)/2)
yele = 0
zele = 0
Xele, Yele, Zele = np.meshgrid(xele, yele, zele)
eles = np.array([Xele.flatten(), Yele.flatten(), Zele.flatten()]).T

In [9]:
params = {}
params['points'] = field
params['c'] = 1540
params['fnum'] = 1
params['alphas'] = np.repeat(np.linspace(-np.pi/8, np.pi/8, na), nele)
params['trefs']  = np.zeros(params['alphas'].shape)
params['refs'] = np.tile(eles, (na, 1))
params['ts'] = 1E-7
params['tstart'] = 0
params['nsamp'] = 1500

In [10]:
bmfrm = pwb.PWBeamformer(**params)

Initializing a PWBeamformer...
  Formatting input parameters...
  Putting shared values...
    Allocating trefs...
    Allocating points...
    Allocating tinds...
    Allocating masks...
    Allocating databuffer...
  Registering beamformer with global indexes
  Filling tables
    Generating masks
    Generating Transmission Tabs
